In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
#from torchview import draw_graph
import matplotlib.pyplot as plt
import numpy as np

### Loading the data

In [10]:
class BuildingDataset(Dataset):
    def __init__(self, path):
        '''Loads the data'''
        # read raster maps
        # self.x = torch.from_numpy()
        # load the generalization operators
        # self.y = torch.from_numpy()
        #self.n_samples = xy.shape[0]
        pass

    def __getitem__(self, index):
        '''Enables indexing'''
        # returns self.x[index], self.y{index}
        pass
        
    def __len__(self):
        '''Enables dataset length calculation'''
        #return self.n_samples 
        pass

path = "../training_samples.nosync/raster"

#dataset = BuildingDataset()
#dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)

#dataiter = iter(dataloader)

### Training pipeline

1) Design model (input, output size, forward pass)
2) Construct loss and optimizer
3) Training loop
     * Forward pass: Compute prediction
     * Backward pass: Compute gradients
     * Update weights

In [2]:
# Metal device enables running the code on the GPU on MacOS (analogous to CUDA)
device = torch.device("mps")

In [8]:
x = torch.ones(5, device=device)
y = torch.ones(5)
y = y.to(device)

z = x+y

# move the tensor back to the CPU for numpy conversion
z = z.to("cpu")

z.numpy()

# requires_grad = True for variables that should be optimized

array([2., 2., 2., 2., 2.], dtype=float32)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First convolutional layer taking 1 input channel (image), 16 output channels, kernel size 3
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        # Second convolutional layer, taking 16 input channels, outputting 32 channels, kernel size 3
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        # Third convolutional layer, taking 32 input channels, outputting 64 channels, kernel size 3
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # A MaxPooling Layer to reduce the spatial dimensions of the output from convolutional layers
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layer taking the flattened output from the last convolutional layer and outputting 10 classes
        # (for CIFAR-10, for example)
        self.fc = nn.Linear(64 * 4 * 4, 10)  # Assuming the input size to the network is 32x32 pixels

    def forward(self, x):
        # Apply convolutions followed by batch normalization, a ReLU activation, and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the output for the fully connected layer
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

# Example of using the model
model = CNN()
print(model)

model.to(device)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1024, out_features=10, bias=True)
)


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
# prediction evaluations should not be part of the computational graph
#with torch.no_grad():
    